In [2]:
import praw
import os
import json
import time

from supabase import create_client, Client

# Supabase setup
url: str = os.environ.get('SUPABASE_WATCHDB_URL')
key: str = os.environ.get('SUPABASE_WATCHDB_SERVICE_ROLE_KEY')
supabase: Client = create_client(url, key)

# Reddit API Credentials
client_id = os.environ.get('REDDIT_APP_ID')
client_secret = os.environ.get('REDDIT_APP_KEY')
user_agent = 'User-Agent:chrono-codex-server:v1 (by /u/ChronoCrawler)'

# Initialize PRAW with your credentials
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent)


Lad wrote a Python script to download Alexa voice recordings, he didn't expect this email.
This post has:
I redesign the Python logo to make it more modern
Automate the boring stuff with python - tinder
Just finished programming and building my own smart mirror in python, wrote all of the code myself and implemented my own voice control and facial recognition features


In [5]:
# The subreddit you want to scrape
subreddit = reddit.subreddit('watchexchange')

# Initialize a list to store data before saving to disk
data_list = []

# Fetch the top posts from the subreddit
top_posts = subreddit.top(time_filter="hour", limit=10)  # Adjust limit as needed


# Push the data collected to Supabase
for post in top_posts:
    post.comments.replace_more(limit=25)  # Load all comments
    # comments = [{'userid': comment.author.name, 'comment': comment.body} for comment in post.comments.list()]
    comments = ' | '.join([f"{comment.author.name}: {comment.body}" for comment in post.comments.list()])

    post_data = {
        'post_id': post.id,
        'author_id': post.author.name,
        'title': post.title,
        'url': post.url,
        'comments': comments
    }
    
    try:
        # Attempt to insert post_data into your Supabase table
        data_insert_response = supabase.table('rqueue').insert(post_data).execute()
    except Exception as e:
        if 'duplicate key value violates unique constraint "rqueue_pkey"' in str(e):
            print(f"Skipping insertion for post_id={post_data['post_id']} as it already exists.")
        else:
            raise


2024-02-06 17:53:19,214:INFO - HTTP Request: POST https://gvvfniijngcyqnwvrbal.supabase.co/rest/v1/rqueue "HTTP/1.1 409 Conflict"


Skipping insertion for post_id=1akq7lk as it already exists.


2024-02-06 17:53:19,456:INFO - HTTP Request: POST https://gvvfniijngcyqnwvrbal.supabase.co/rest/v1/rqueue "HTTP/1.1 201 Created"
2024-02-06 17:53:19,743:INFO - HTTP Request: POST https://gvvfniijngcyqnwvrbal.supabase.co/rest/v1/rqueue "HTTP/1.1 201 Created"
2024-02-06 17:53:20,017:INFO - HTTP Request: POST https://gvvfniijngcyqnwvrbal.supabase.co/rest/v1/rqueue "HTTP/1.1 201 Created"
2024-02-06 17:53:20,316:INFO - HTTP Request: POST https://gvvfniijngcyqnwvrbal.supabase.co/rest/v1/rqueue "HTTP/1.1 201 Created"
2024-02-06 17:53:20,715:INFO - HTTP Request: POST https://gvvfniijngcyqnwvrbal.supabase.co/rest/v1/rqueue "HTTP/1.1 201 Created"
2024-02-06 17:53:20,948:INFO - HTTP Request: POST https://gvvfniijngcyqnwvrbal.supabase.co/rest/v1/rqueue "HTTP/1.1 201 Created"
2024-02-06 17:53:21,310:INFO - HTTP Request: POST https://gvvfniijngcyqnwvrbal.supabase.co/rest/v1/rqueue "HTTP/1.1 201 Created"
2024-02-06 17:53:21,584:INFO - HTTP Request: POST https://gvvfniijngcyqnwvrbal.supabase.co/rest/v

In [8]:
import openai
import json
import jsonschema

# Load your OpenAI API key from an environment variable or directly
openai_api_key = os.environ.get('OPENAI_API_CHRONO_KEY')

openai.api_key = openai_api_key

with open('query_schema.json') as f:
    output_schema_str = f.read()

# Fetch data from Supabase queue
try:
    queue_data = supabase.table('rqueue').select('*').eq('processed', False).limit(1).execute()
    if len(queue_data.data) > 1:
        raise Exception("lll")
    if queue_data.data:
        for item in queue_data.data:
            # Convert the item to JSON string
            item_json = json.dumps(item)

            prompt = f"Given the data: {item_json}, construct a JSON object that adheres to the specified output schema. Here is the output schema: {output_schema_str}"
            try:
                response = openai.Completion.create(
                    model="gpt-3.5-turbo-0125",
                    prompt=prompt,
                    temperature=0.5,  # Adjust as needed
                    max_tokens=1024,  # Adjust based on your expected output size
                    top_p=1.0,
                    frequency_penalty=0.0,
                    presence_penalty=0.0
                )
                print(response.choices[0].text)
            except Exception as e:
                print(f"An error occurred: {e}")

    else:
        print("No data found in the queue.")
except Exception as e:
    print(f"Failed to fetch data from Supabase: {e}")


2024-02-06 18:18:58,543:INFO - HTTP Request: GET https://gvvfniijngcyqnwvrbal.supabase.co/rest/v1/rqueue?select=%2A&processed=eq.False&limit=1 "HTTP/1.1 200 OK"


An error occurred: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

